In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import s3fs, pandas as pd,os
import pickle
import yaml
import os

In [35]:
!rm -rf /home/ec2-user/SageMaker/FSI_SMS_TO_CONVERSION/predictions/

In [5]:
fs = s3fs.S3FileSystem()
flat_demography = []
path = "s3://data-lake-v2/processed_batch_data/clean/flat_demography/year=2020/month=03/"
for file in fs.ls(path):
    flat_demography.append(file)

In [ ]:
data = pd.read_parquet("s3://"+self.path, engine = 'pyarrow', columns = self.input_col)
data = data.reindex(columns = self.input_col)

In [9]:
for item in flat_demography[:1]:
    print (item)
    data = pd.read_parquet("s3://"+item, engine = 'pyarrow')
    print (data.head())

data-lake-v2/processed_batch_data/clean/flat_demography/year=2020/month=03/part-00000-083d9589-8516-4f69-938b-080c6b1128cf-c000.snappy.parquet
                             msisdn                                tap_id  \
0  a511b7332140982683bbce5d1723bfc8  65e9975a-a3f4-46b4-91fc-98a968e00e69   
1  74a8e2ae0b0d8e8ad9fe2fd28583ba63  637841cb-9bec-452b-97d1-047e5342e447   
2  49140903bb0cfed4b29424cdced7c16e  ae3136f3-ee6f-40f4-aa5b-061db1c0023f   
3  744b982414f3616d4400a81e4b10bb0a  de2f924a-0244-4ce7-9ccd-92958e332b5c   
4  9e7befe0be23437db41a3b653b27e76a  963254d5-e476-43ba-a112-bf6c632fd05d   

  location_region location_state location_lga device_manufacturer  \
0            None           None         None                None   
1            None           None         None                None   
2      North West        katsina      katsina             samsung   
3            None           None         None                None   
4      North West        katsina      katsina    

In [36]:
!sudo sysctl -w vm.drop_caches=3

vm.drop_caches = 3


In [32]:
#input_col = ['msisdn', 'spend_total', 'age', 'device_manufacturer', 'gender', 'customer_value', 'customer_class', 'location_state', 'device_type']
features = yaml.safe_load(open("Attributes_yaml/features.yml"))
input_col = features['input_col']
fs = s3fs.S3FileSystem()
#path to the data
path = "s3://data-lake-v2/processed_batch_data/clean/flat_demography/year=2020/month=03/"
directory = os.getcwd()
pred_path = os.path.join(directory,'predictions')

def score(data_path):
    counter = 0
    for file in fs.ls(data_path):
        print (file)
        data = pd.read_parquet("s3://"+file, engine='pyarrow', columns = input_col)
        msisdn = data.msisdn
        data = data.drop(['msisdn'], axis=1)
        num_features = data.select_dtypes(include=["number"]).columns
        cat_features = data.select_dtypes(exclude=["number"]).columns
        for item in cat_features:
            data[item] = data[item].fillna(data[item].value_counts().index[0])
        for item in num_features:
            data[item] = data[item].fillna(data[item].mean())
        with open("/home/ec2-user/SageMaker/FSI_SMS_TO_CONVERSION/model_and_pipeline/cvr2_pipe.pkl", 'rb') as file:
            pipe = pickle.load(file)
        pipe_data = pipe.transform(data)
        
        with open("/home/ec2-user/SageMaker/FSI_SMS_TO_CONVERSION/model_and_pipeline/model_log_reg.pkl", 'rb') as file:
            model = pickle.load(file)
        predictions = np.round(model.predict_proba(pipe_data)[:,1],3)
        out = 'out'+ str(counter) + '.csv'
        pd.DataFrame({'msisdn':msisdn,'results':predictions}).to_csv(pred_path + '/' + out,index=False,
                                                                     sep=',',header=['msisdn','score'])
        counter +=1
        print('Done. Check the predictions path')


In [ ]:
score(path)

In [ ]:
!aws s3 cp /home/ec2-user/SageMaker/FSI_SMS_TO_CONVERSION/predictions/. s3://datateam-ml/predictions/fsi_sms_cvr_pytorch/ --recursive